# Generate Synthetic Data

## Constants

In [1]:
PRIMER = ""
GPT_4O_MINI = "gpt-4o-mini"
TEMPERATURE = 1
TOP_P = 0.1

## Imports

In [2]:
from langchain.prompts import PromptTemplate
import openai
from dotenv import dotenv_values
import pandas as pd
import numpy as np

config = dotenv_values("../.env")


In [3]:
client = openai.OpenAI(api_key=config["API_KEY"], max_retries=5)

## Auxiliar Functions

In [5]:
def predict_gpt(text: str, model=GPT_4O_MINI, temperature=TEMPERATURE, primer=PRIMER, response_format = {"type": "text"}):
    """
    Generates a text completion using the GPT model.

    Args:c
        text (str): The user's input text.
        model: The GPT model to use for text generation.
        temperature: The temperature parameter for text generation.

    Returns:
        The generated text completion.
    """
    # print(text)
    result = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": primer},
            {"role": "user", "content": text},
        ],
        temperature=temperature,
        response_format=response_format,
        seed=42,
        top_p=TOP_P
    )

    return result


In [6]:
def format_prompt(template, subject, sub_subject_title):
    identify_elements_prompt = PromptTemplate(input_variables=["subject", "sub_subject_title"], template=template)
    prompt = identify_elements_prompt.format(subject=subject, sub_subject_title=sub_subject_title)
    return prompt

In [7]:
def generate_jsonl_input_data(data, template, model=GPT_4O_MINI, temperature=TEMPERATURE, top_p=TOP_P, primer=PRIMER):
    """
    Generates JSONL input data for batch processing with OpenAI API.

    Args:
        data (pd.DataFrame): DataFrame containing subject and title information.
        template (str): Prompt template string.
        model (str): GPT model to use. Defaults to GPT_4O_MINI.
        temperature (float): Temperature for text generation. Defaults to TEMPERATURE.
        top_p (float): Top-p sampling parameter. Defaults to TOP_P.
        primer (str): System message content. Defaults to PRIMER.

    Returns:
        list: List of dictionaries containing input data for each request.
    """
    inputs = []
    for index, row in data.iterrows():
        custom_id = f"request-{index}"
        subject = row["subject"]
        sub_subject_title = row["title"]
        
        prompt = format_prompt(template, subject, sub_subject_title)
        
        messages = [
            {"role": "system", "content": primer},
            {"role": "user", "content": prompt}
        ]
        
        input_data = {
            "custom_id": custom_id,
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": model,
                "messages": messages,
                "max_tokens": 2000,
                "temperature": temperature,
                "top_p": top_p
            }
        }
        inputs.append(input_data)
    
    return inputs

## Load data_subjects

In [8]:
data = pd.read_csv('../data/master_data/master_data.csv')

In [9]:
print(f"Data shape: {data.shape}")
data['title'] = data['title'].str.lower()
data = data.drop_duplicates(subset=['title'])
print(f"Data shape: {data.shape}")


Data shape: (4964, 5)
Data shape: (4304, 5)


In [13]:
template = """Você é um especialista em educação, conhecido por criar materiais didáticos claros e acessíveis. Sua tarefa é elaborar um resumo didático, voltado para estudantes do ensino médio, sobre o tema "{sub_subject_title}" em {subject}. Use uma linguagem simples, exemplos práticos e organize o conteúdo de forma que os alunos entendam facilmente o tema."""

## Generate Input Data

In [18]:
data['prompt'] = generate_jsonl_input_data(data, template)

In [25]:
import json
# File path where you want to save the JSONL file
file_path = '../gpt_data/input_data/input_data.jsonl'

# Writing to a JSONL file
with open(file_path, 'w') as f:
    for entry in data['prompt']:
        json_line = json.dumps(entry, ensure_ascii=False)
        f.write(json_line + '\n') 

In [26]:
with open(file_path, 'r') as f:
    lines = f.readlines()

# Calculate the number of lines per batch
num_batches = 4
batch_size = len(lines) // num_batches

# Split lines into batches
for i in range(num_batches):
    start_index = i * batch_size
    end_index = start_index + batch_size if i < num_batches - 1 else len(lines)  # Ensure the last batch gets any remaining lines
    with open(f'../gpt_data/input_data/input_data_{i+1}.jsonl', 'w') as out_file:
        out_file.writelines(lines[start_index:end_index])

## Upload File to OpenAi

In [28]:
batch_input_file = []

In [29]:
for i in range(num_batches):
    batch_input_file.append(client.files.create(
        file=open(f'../gpt_data/input_data/input_data_{i+1}.jsonl', "rb"),
        purpose="batch"
    ))
    print(f"File {i+1} uploaded")

File 1 uploaded
File 2 uploaded
File 3 uploaded
File 4 uploaded


In [31]:
batch_input_file[0]

FileObject(id='file-lJ8suwcrAtTZrbUNPCLlvLXh', bytes=654259, created_at=1725508290, filename='input_data_1.jsonl', object='file', purpose='batch', status='processed', status_details=None)

## Process Batch

In [33]:

for i in range(num_batches):
    print(f"{i+1} id: {batch_input_file[i].id}")


1 id: file-lJ8suwcrAtTZrbUNPCLlvLXh
2 id: file-Dert7iT5cfFqNgKn67lY7jFC
3 id: file-bT0MDwhZIcDI8FsqzNHamQJ2
4 id: file-1mzQo9Nil6gsYZXpU4xDJjWQ


In [5]:
batch_input_file_id = "file-1mzQo9Nil6gsYZXpU4xDJjWQ"

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "geração de dados, teste v2 todos dados"
    }
)

Batch(id='batch_NEwqGLKyB6SMatPnTOLVKYFS', completion_window='24h', created_at=1725594345, endpoint='/v1/chat/completions', input_file_id='file-1mzQo9Nil6gsYZXpU4xDJjWQ', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1725680745, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'geração de dados, teste v2 todos dados'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [ ]:
## Prepare batch input 

In [ ]:
# batch_input_file_id = batch_input_file.id

# client.batches.create(
#     input_file_id=batch_input_file_id,
#     endpoint="/v1/chat/completions",
#     completion_window="24h",
#     metadata={
#       "description": "nightly eval job"
#     }
# )

In [ ]:
# load /home/letrus/Documentos/Personal_Projects/soft_eng2/batch_mZZXQ9LwO5Wt1ZDF1NDqJmbp_output.jsonl
output_data = pd.read_json('/home/letrus/Documentos/Personal_Projects/soft_eng2/batch_MHvUtGOukK9sa0qe5YdllcES_output.jsonl', lines=True)

In [ ]:
output_data['response'].iloc[0]['body']['choices'][0]['message']['content']

'# Introdução à Aritmética: Operações Básicas\n\n## O que é Aritmética?\n\nA Aritmética é uma das áreas da Matemática que lida com números e as operações que podemos realizar com eles. As operações básicas da Aritmética são:\n\n1. **Adição (+)**\n2. **Subtração (−)**\n3. **Multiplicação (×)**\n4. **Divisão (÷)**\n\nEssas operações são fundamentais para resolver problemas do dia a dia e são a base para aprender matemática mais avançada.\n\n## 1. Adição\n\nA adição é a operação de juntar ou somar quantidades. \n\n### Exemplo:\nSe você tem 3 maçãs e ganha mais 2, quantas maçãs você tem agora?\n\n**Cálculo:**\n3 + 2 = 5\n\nPortanto, você tem 5 maçãs.\n\n### Propriedades da Adição:\n- **Comutativa:** A ordem dos números não altera o resultado. Exemplo: 2 + 3 = 3 + 2.\n- **Associativa:** A forma como agrupamos os números não altera o resultado. Exemplo: (1 + 2) + 3 = 1 + (2 + 3).\n\n## 2. Subtração\n\nA subtração é a operação de retirar ou diminuir uma quantidade de outra.\n\n### Exemplo:\nS

In [ ]:
json.loads(output_data['response'].iloc[0])

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:
output_data['response_content'] = output_data['response'].apply(lambda x: x['body']['choices'][0]['message']['content'])

In [ ]:
# join the output data with data by idxs 
output_data['idx'] = output_data.index
data['idx'] = data.index

# join
merged_data = data.merge(output_data, on='idx')

/tmp/ipykernel_13607/3755428332.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['idx'] = data.index


In [ ]:
merged_data[['title', 'subject', 'response_content']].to_csv('geral_output.csv', index=False)

In [ ]:
merged_data

,title,url,content,subject,sub-subject,tokens,idx,id,custom_id,response,error,response_content
0,Introdução Aritmética I Operações Básicas,https://resumos.mesalva.com/introducao-aritmet...,"Ao lado da física e da química, a matemática é...",Matematica,introducao-aritmetica-operacoes-basicas,1656,0,batch_req_oHSv36ra5jyBRAvXfzi7lsJq,request-0,"{'status_code': 200, 'request_id': 'be6626008d...",NaN,# Introdução à Aritmética: Operações Básicas\n...
1,Adição e Subtração de Naturais,https://resumos.mesalva.com/adicao-e-subtracao...,É caracterizada pela soma de duas ou mais “coi...,Matematica,adicao-e-subtracao-de-naturais-2,604,1,batch_req_HbRaAPKOSeFhHUVT3SuQNnCy,request-1,"{'status_code': 200, 'request_id': 'f95d9a65b1...",NaN,# Resumo Didático: Adição e Subtração de Númer...
2,Multiplicação e Divisão de Naturais,https://resumos.mesalva.com/multiplicacao-e-di...,Essa operação é uma forma mais fácil de realiz...,Matematica,multiplicacao-e-divisao-de-naturais,547,2,batch_req_NoTxiwmwxVbu0O6P5jwZU91Z,request-2,"{'status_code': 200, 'request_id': 'e275b2e093...",NaN,# Resumo Didático: Multiplicação e Divisão de ...
3,Máximo Divisor Comum (MDC) e os Números Primos,https://resumos.mesalva.com/maximo-divisor-com...,"Ao planejar a sua festinha de aniversário, voc...",Matematica,maximo-divisor-comum-mdc-numeros-primos,974,3,batch_req_OIHusrKnnBWcL4RMmKoMi9MZ,request-3,"{'status_code': 200, 'request_id': 'c46998664f...",NaN,# Máximo Divisor Comum (MDC) e Números Primos\...
4,Mínimo Múltiplo Comum – MMC,https://resumos.mesalva.com/minimo-multiplo-co...,"Novamente, essa técnica é aplicada em dois ou ...",Matematica,minimo-multiplo-comum-mmc,629,4,batch_req_naiXy3JGZ1vvuE6H10PfRwA9,request-4,"{'status_code': 200, 'request_id': '9b13bc1f41...",NaN,# Mínimo Múltiplo Comum (MMC)\n\n## O que é o ...
...,...,...,...,...,...,...,...,...,...,...,...,...
938,A Missão Artística Francesa e a Arte Acadêmica,https://resumos.mesalva.com/missao-artistica-f...,A mudança da corte portuguesa para o Rio de Ja...,Artes,missao-artistica-francesa-arte-academica,624,940,batch_req_o6ihcg3LP8nrq0QPGluqq999,request-942,"{'status_code': 200, 'request_id': '3279843a40...",NaN,# Movimento Antropofágico nas Artes\n\n## O qu...
939,Semana de Arte Moderna,https://resumos.mesalva.com/semana-arte-moderna/,O gosto pela Arte Acadêmica apenas seria abala...,Artes,semana-arte-moderna,958,941,batch_req_VZFh8kEOCDRMfnhozfaaMdTU,request-943,"{'status_code': 200, 'request_id': 'f2bf3f5a67...",NaN,# Arte Contemporânea Brasileira\n\n## O que é ...
940,Movimento Antropofágico,https://resumos.mesalva.com/movimento-antropof...,Dificilmente imaginaríamos o surgimento do Mov...,Artes,movimento-antropofagico,525,942,batch_req_p8AK196DkFf31VoLlkT1AhJf,request-944,"{'status_code': 200, 'request_id': '832ff98f2c...",NaN,# Cinema e Literatura: Um Diálogo\n\n## Introd...
941,Arte Contemporânea Brasileira,https://resumos.mesalva.com/arte-contemporanea...,"De um modo geral, quando falamos sobre Arte Co...",Artes,arte-contemporanea-brasileira,1533,943,batch_req_VFwE4gbyqvgIPlrlmmxYjJqN,request-945,"{'status_code': 200, 'request_id': '5fed5c1bda...",NaN,# Leituras Adaptadas em Artes\n\n## O que são ...


In [ ]:
merged_data.to_csv('geral_output_v0.csv', index=False)